# Word Embeddings em um Dataset Aberto

## Terceiro Exercício Prático


Neste TP você deve explorar e criar os embeddings utilizando uma base de dados de sua preferência. Uma sugestão para encontrar bases interessantes seria a partir das competições do [Kaggle](https://www.kaggle.com/).

Para essa prática, solicito que o aluno prepare a base de dados e gere os embeddings utilizando obrigatoriamente o algoritmo Word2Vec. Dependendo do contexto da base, você pode utilizar o Doc2Vec em vez do Word2Vec ou ambos. 

As 3 etapas descritas abaixo devem ser seguidas obrigatoriamente:

1. Preparação da base de dados assim como visto na prática anterior.
2. Execução do Modelo Word2Vec usando o Gensim, ou outra implementação similar.
3. Teste do seu embedding assim como foi realizado na [demo](https://github.com/gesteves91/nlp/blob/master/notebooks/06-word2vec.ipynb).


Para o trabalho foi capturado os tweets sobre o ex-presidente Lula no dia em que recebeu autorização para deixar a cadeia. Especificamos o dia 08/11/2019 (data em que ele saiu), restringindo a localização na região de Belo Horizonte, num raio de 10.000 km.

Documentação:<br> 
https://tweepy.readthedocs.io/en/latest/index.html

In [120]:
# bibliotecas
import numpy as np
import pandas as pd
import tweepy
import nltk
from nltk.tokenize import word_tokenize
import re

In [2]:
# leitura das chaves 
with open('twitter-tokens.txt', 'r') as tfile:
    consumer_key = tfile.readline().strip('\n')
    consumer_secret = tfile.readline().strip('\n')
    access_token = tfile.readline().strip('\n')
    access_token_secret = tfile.readline().strip('\n')

In [3]:
# variaveis para fazer o login
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [18]:
# Query passando a busca e excluindo os retweets.
query_search = 'Lula' + '-filter:retweets'

#Gerando o cursor de busca.
cursor_tweets = tweepy.Cursor(api.search, q=query_search, tweet_mode='extended',lang="pt", 
                              since= "2019-11-07", until= "2019-11-09",
                              geocode='-19.9026615,-44.1041363,10000km').items(10000)

In [111]:
tw = []
for tweet in cursor_tweets:
    tw.append([tweet.created_at, tweet.full_text])

In [20]:
df = pd.DataFrame(tw, columns=['Data','Tweet'])

In [21]:
# Convertendo a data
df['Data'] = pd.to_datetime(df['Data'])
df['Data'] = df['Data'].dt.strftime('%d-%m-%Y')

In [22]:
# Salvando em um arquivo csv.
df.to_csv('bh3.csv')

In [23]:
df.shape

(6750, 2)

# Preparando a base de dados

In [73]:
df_bh = pd.read_csv('bh3.csv')

In [74]:
df_bh.shape

(6750, 3)

In [75]:
df_bh.head()

,Unnamed: 0,Data,Tweet
0,0,08-11-2019,Lula virou comediante na cadeia e saiu fazendo...
1,1,08-11-2019,Hoj o Lula transa com o super pênis dele https...
2,2,08-11-2019,@ana_claudiinha até o Lula tá beijando e você ...
3,3,08-11-2019,@ggreenwald Bravo! Bravo! Você tem grande impo...
4,4,08-11-2019,"Vou me ausentar desse site, quando pararem de ..."


In [136]:
df = df_bh[['Data', 'Tweet']].copy()

In [137]:
df.head()

,Data,Tweet
0,08-11-2019,Lula virou comediante na cadeia e saiu fazendo...
1,08-11-2019,Hoj o Lula transa com o super pênis dele https...
2,08-11-2019,@ana_claudiinha até o Lula tá beijando e você ...
3,08-11-2019,@ggreenwald Bravo! Bravo! Você tem grande impo...
4,08-11-2019,"Vou me ausentar desse site, quando pararem de ..."


# Tokenização

In [103]:
from nltk.tokenize import sent_tokenize

In [106]:
df['Sent_tok'] = df.apply(lambda x: sent_tokenize(x['Tweet']), axis=1)

In [108]:
# Tokenizando as frases.
df['Tokens'] = df.apply(lambda x: word_tokenize(x['Tweet']), axis=1)

In [109]:
# Convertendo para minúsculos.
df['Tokens'] = df.apply(lambda row: map(lambda x:x.lower(), row['Tokens']), axis=1)

# Lematização

In [115]:
stemmer = nltk.stem.WordNetLemmatizer()

In [119]:
df

,Data,Tweet,Sent_tok,Tokens
0,08-11-2019,Lula virou comediante na cadeia e saiu fazendo...,[Lula virou comediante na cadeia e saiu fazend...,<map object at 0x7fc506974dd8>
1,08-11-2019,Hoj o Lula transa com o super pênis dele https...,[Hoj o Lula transa com o super pênis dele http...,<map object at 0x7fc506974e48>
2,08-11-2019,@ana_claudiinha até o Lula tá beijando e você ...,[@ana_claudiinha até o Lula tá beijando e você...,<map object at 0x7fc506974eb8>
3,08-11-2019,@ggreenwald Bravo! Bravo! Você tem grande impo...,"[@ggreenwald Bravo!, Bravo!, Você tem grande i...",<map object at 0x7fc506974f28>
4,08-11-2019,"Vou me ausentar desse site, quando pararem de ...","[Vou me ausentar desse site, quando pararem de...",<map object at 0x7fc506974f98>
5,08-11-2019,Sextei com Lula livre https://t.co/E9t0p9VYLs,[Sextei com Lula livre https://t.co/E9t0p9VYLs],<map object at 0x7fc506976048>
6,08-11-2019,O líderes do mundo todo comemorando a liberdad...,[O líderes do mundo todo comemorando a liberda...,<map object at 0x7fc5069760b8>
7,08-11-2019,"todo mundo sabe q o lula ta solto, se emociona...","[todo mundo sabe q o lula ta solto, se emocion...",<map object at 0x7fc506976128>
8,08-11-2019,Será que a Débora dos falsetes já lançou o do ...,[Será que a Débora dos falsetes já lançou o do...,<map object at 0x7fc506976198>
9,08-11-2019,Ah os refrescos..... Tô aqui mto feliz colocan...,[Ah os refrescos..... Tô aqui mto feliz coloca...,<map object at 0x7fc506976208>
